# Basic Synthetic Data Generation with DeepBridge

This tutorial demonstrates how to generate synthetic data using the DeepBridge library. I'll walk you through creating synthetic datasets with different methods and comparing their results.

## Overview

In this demonstration, we'll:
1. Create a sample dataset with mixed data types
2. Generate synthetic versions using three different methods:
   - Gaussian Copula
   - CTGAN (Conditional Tabular GAN)
   - UltraLight Generator
3. Evaluate and compare the quality of each method
4. Visualize the differences between original and synthetic data

## Understanding the Different Methods

Each synthetic data generation method has its unique characteristics:

### Gaussian Copula
- Statistical method that preserves the marginal distributions and correlations between features
- Good balance between quality and computational efficiency
- Works well for numerical data with linear relationships
- Medium memory requirements

### CTGAN (Conditional Tabular GAN)
- Neural network-based approach using Generative Adversarial Networks
- Can capture complex, non-linear relationships in the data
- Highest quality for capturing complex patterns
- More computationally intensive and requires more memory
- Longer training time

### UltraLight Generator
- Simplest and fastest approach with minimal memory requirements
- Uses basic statistical modeling rather than complex ML models
- Excellent for large datasets or limited computational resources
- Quality may be lower for complex relationships

## Example Implementation

Let's look at the code to implement these methods:

In [1]:
# pip install deepbridge --upgrade


In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer





import sys
import os

sys.path.append(os.path.expanduser("~/projetos/DeepBridge"))



from deepbridge.core.db_data import DBDataset
# from deepbridge.synthetic import Synthesize
from deepbridge.core.experiment import Experiment


# from deepbridge.validation.wrappers import (
#     RobustnessSuite, UncertaintySuite, 
# )

# from deepbridge.utils.robustness import run_robustness_tests
# from deepbridge.utils.uncertainty import run_uncertainty_tests
# from deepbridge.utils.resilience import run_resilience_tests
# from deepbridge.utils.hyperparameter import run_hyperparameter_tests
#---------------------------------------------------------
# Preparação de dados com cuidado especial 
#---------------------------------------------------------


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

# Gerar dados sintéticos com duas classes
X, y = make_classification(n_samples=10000, n_features=20, n_classes=2, random_state=42)
X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(20)])
y = pd.Series(y)


# Resetar índices para garantir alinhamento limpo
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

# Dividir dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)







# Resetar índices novamente após a divisão
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)


X_train.to_parquet("/home/guhaase/projetos/deepbridge_prod/artefatos/X_train.parquet")
X_test.to_parquet("/home/guhaase/projetos/deepbridge_prod/artefatos/X_test.parquet")
y_train_df = pd.DataFrame({'target': y_train})
y_test_df = pd.DataFrame({'target': y_test})
y_train_df.to_parquet("/home/guhaase/projetos/deepbridge_prod/artefatos/y_train.parquet")
y_test_df.to_parquet("/home/guhaase/projetos/deepbridge_prod/artefatos/y_test.parquet")


# Criar DataFrames de treino e teste com nomes explícitos de colunas
train_df = X_train.copy()
train_df['target'] = y_train
test_df = X_test.copy()
test_df['target'] = y_test

import pickle
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

with open("/home/guhaase/projetos/deepbridge_prod/artefatos/random_forest_model.pickle", "wb") as f:
    pickle.dump(model, f)

with open("/home/guhaase/projetos/deepbridge_prod/artefatos/random_forest_model.pickle", "wb") as f:
    pickle.dump(model, f)

# Criar objeto de dataset

dataset = DBDataset(
    train_data=train_df,
    test_data=test_df,
    target_column='target',
    model=model
)


# Criar e executar o experimento
experiment = Experiment(
      dataset=dataset,
      experiment_type="binary_classification",
      tests=["robustness"],
      feature_subset=['feature_0', 'feature_1']
  )

results = experiment.run_tests("full")


results.save_html("robustness", "report_robustness.html")
# results.save_html("uncertainty", "report_uncertainty.html")
# results.save_html("resilience", "report_resilience.html")

Generating robustness report to: /home/guhaase/projetos/DeepBridge/examples/report_robustness.html
Using templates directory: /home/guhaase/projetos/aval/lib/python3.12/site-packages/deepbridge/templates/reports
Template file exists: /home/guhaase/projetos/aval/lib/python3.12/site-packages/deepbridge/templates/reports/robustness/report.html
Successfully read template file (size: 148504 bytes)
Transforming robustness data structure...
Raw structure keys: ['primary_model', 'alternative_models', 'config', 'experiment_type']
Used deep copy to convert results
Found 'primary_model' key, extracting data...
Processing alternative models data...
Processing alternative model: GLM_CLASSIFIER
Processing alternative model: GBM
Report data structure after transformation:
- primary_model: <class 'dict'>
- alternative_models: <class 'dict'>
- config: <class 'dict'>
- experiment_type: <class 'str'>
- base_score: <class 'float'>
- raw: <class 'dict'>
- quantile: <class 'dict'>
- feature_importance: <cla

'/home/guhaase/projetos/DeepBridge/examples/report_robustness.html'

In [3]:
experiment.initial_results

{'config': {'tests': ['robustness'],
  'verbose': False,
  'dataset_info': {'n_samples': 10000, 'n_features': 20, 'test_size': 0.2}},
 'models': {'primary_model': {'name': 'primary_model',
   'type': 'RandomForestClassifier',
   'metrics': {'accuracy': 0.982,
    'roc_auc': 0.9951927693489533,
    'f1': 0.9819959474671671,
    'precision': 0.982151158739503,
    'recall': 0.982},
   'hyperparameters': {'bootstrap': True,
    'ccp_alpha': 0.0,
    'class_weight': None,
    'criterion': 'gini',
    'max_depth': None,
    'max_features': 'sqrt',
    'max_leaf_nodes': None,
    'max_samples': None,
    'min_impurity_decrease': 0.0,
    'min_samples_leaf': 1,
    'min_samples_split': 2,
    'min_weight_fraction_leaf': 0.0,
    'n_estimators': 100,
    'n_jobs': None,
    'oob_score': False,
    'random_state': 42,
    'verbose': 0,
    'warm_start': False},
   'feature_importance': {'feature_0': 0.409836080111809,
    'feature_1': 0.2543759818918691,
    'feature_19': 0.0944092282696562,
  

In [4]:
results.

SyntaxError: invalid syntax (3562696729.py, line 1)

In [ ]:
experiment.initial_results

In [ ]:
results.results['uncertainty'].results

In [ ]:
experiment.initial_results['models']['primary_model']['type']

results.results['robustness'].results['primary_model']